# Quantum Random Number Generation Workbook

**What is this workbook?**
A workbook is a collection of problems, accompanied by solutions to them. 
The explanations focus on the logical steps required to solve a problem; they illustrate the concepts that need to be applied to come up with a solution to the problem, explaining the mathematical steps required. 

Note that a workbook should not be the primary source of knowledge on the subject matter; it assumes that you've already read a tutorial or a textbook and that you are now seeking to improve your problem-solving skills. You should attempt solving the tasks of the respective kata first, and turn to the workbook only if stuck. While a textbook emphasizes knowledge acquisition, a workbook emphasizes skill acquisition.

This workbook describes the solutions to the problems offered in the [Random Number Generation Tutorial](./RandomNumberGenerationTutorial.ipynb). 
Since the tasks are offered as programming problems, the explanations also cover some elements of Q# that might be non-obvious for a first-time user.

**What you should know for this workbook**

You should be familiar with the following concepts before tackling the Quantum Random Number Generation Tutorial (and this workbook):

1. The concept of qubit and measurement
2. Single qubit gates

## <span style="color:blue">Exercise 1</span>: Generate a single random bit

**Input:** None.

**Goal:** Generate a $0$ or $1$ with equal probability.

<details>
    <summary><strong>Need a hint? Click here</strong></summary>
    Use the allocated qubit, apply a quantum gate to it, measure it and use the result to return a $0$ or $1$.
</details>

**Stretch goal:** Can you find a different way to implement this operation?

<details>
    <summary><strong>Need a hint? Click here</strong></summary>
    What are the different quantum states that produce $0$ and $1$ measurement results with the same probability? How would measuring the qubit in a different basis change the result? 
</details>


### Solution

The state of single qubit can be represented two-dimensional column vector $\begin{bmatrix} \alpha\\ \beta \end{bmatrix}$. Where $\alpha$ and $\beta$ are complex numbers. When we measure the qubit, we get either 0 with probability $|\alpha|^2$ (or) 1 with probability $|\beta|^2$. Essentially we can control probablity of measurement outcome by setting right amplitudes of basis states $\alpha$ and $\beta$. 

When we initilize qubit, amplitudes $\alpha$ and $\beta$ are 1 and 0 respectively. Now our goal is set equal amplitudes for $\alpha$ and $\beta$ for absolute randomness. We can achieve that by simply applying Hadamard gate on the base state.

$$
H|0\rangle=
\frac{1}{\sqrt{2}}\begin{bmatrix}
   1 & 1 \\
   1 & -1
  \end{bmatrix}
 \begin{bmatrix}
   1\\
   0\\
  \end{bmatrix}
=
\frac{1}{\sqrt{2}}\begin{bmatrix}
   1 \cdot 1 + 1 \cdot 0 \\
   1 \cdot 1 + (-1) \cdot 0
  \end{bmatrix}
=
  \frac{1}{\sqrt{2}}\begin{bmatrix}
   1\\
   1
  \end{bmatrix}
$$

Now, both 0 and 1 outcomes are with equal probablity of $|\frac{1}{\sqrt{2}}|^2 = \frac{1}{2}$.

> Note: Since probablity is mod squared of amplitude, we will get same randomness by applying Hadamard gate on $|1\rangle$. Try it out as optional exercise.

In [2]:
%kata T1_RandomBit

operation RandomBit () : Int {
    // Allocate single qubit
    use q = Qubit();
    
    // Set qubit in superposition state
    H(q);
    
    // Measuring state of qubit and return integer value of result
    return (M(q) == Zero) ? 0 | 1;
}

Testing one random bit generation...
Test passed


Success!

## <span style="color:blue">Exercise 2</span>: Generate a random two-bit number

Now that you can generate a single random bit, you can use that logic to create random multi-bit numbers. Let's try first to make a two-bit number by combining two randomly generated bits.

**Input:** None.

**Goal:** Generate a random number in the range $[0, 3]$ with an equal probability of getting each of the four numbers.

**Stretch goal:** Can you do this without allocating qubits in this operation?

<details>
    <summary><strong>Need a hint? Click here</strong></summary>
    Remember that you can use the previously defined operations.
</details>

### Solution

Reusing `RandomBit` operation from [Exercise 1](#Exercise-1:-Generate-a-single-random-bit), calling twice to generate random two-bit number.

In [7]:
%kata T2_RandomTwoBits

operation RandomBit () : Int {
    // Allocate single qubit
    use q = Qubit();
    
    // Set qubit in superposition state
    H(q);
    
    // Measuring state of qubit and return integer value of result
    return (M(q) == Zero) ? 0 | 1;
}

operation RandomTwoBits () : Int {
    return 2 * RandomBit() + RandomBit();
}

Expecting only one Q# operation in code. Using the first one
Testing two random bits generation...
Unexpected number generated. Expected values from 0 to 3, generated -1
Unexpected number generated. Expected values from 0 to 3, generated -1
Unexpected number generated. Expected values from 0 to 3, generated -1


Failed to generate sufficiently random integer
Try again!


## <span style="color:blue">Exercise 3</span>: Generate a number of arbitrary size

Let's take it a step further and generate an $N$-bit number. 

> Remember that you can use previously defined operations in your solution.

**Input:** An integer $N$ ($1 \le N \le 10$).

**Goal:** Generate a random number in the range $[0, 2^N - 1]$ with an equal probability of getting each of the numbers in this range.

> Useful Q# documentation: 
> * [`for` loops](https://docs.microsoft.com/azure/quantum/user-guide/language/statements/iterations), 
> * [mutable variables](https://docs.microsoft.com/azure/quantum/user-guide/language/typesystem/immutability), 
> * [exponents](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.math.powi).

### Solution



In [4]:
%kata T3_RandomNBits 

open Microsoft.Quantum.Math;

operation RandomBit () : Int {
    // Allocate single qubit
    use q = Qubit();
    
    // Set qubit in superposition state
    H(q);
    
    // Measuring state of qubit and return integer value of result
    return (M(q) == Zero) ? 0 | 1;
}

operation RandomNBits (N : Int) : Int {
    // Set max as 2 ^ N - 1
    let max = PowI(2, N) - 1;

    // Declare result variable with mutable binding
    mutable result = 0;

    repeat {
        set result = 0;

        for index in 1 .. N {
            set result = (result * 2) + RandomBit();
        }
    } until (result <= max);

    return result;
}

Expecting only one Q# operation in code. Using the first one
Testing N = 1...
Unexpected number generated. Expected values from 0 to 1, generated -1
Unexpected number generated. Expected values from 0 to 1, generated -1
Unexpected number generated. Expected values from 0 to 1, generated -1


Failed to generate sufficiently random integer
Try again!


## <span style="color:blue">Exercise 4</span>: Generate a weighted bit!

In each of the above exercises, all generated numbers were equally likely. Now let's create an operation that will return a random bit with different probabilities of outcomes. 

> Remember that by setting amplitudes of basis states $\alpha$ and $\beta$, we can control the probability of getting measurement outcomes $0$ and $1$ when the qubit is measured.

**Input:** 
A floating-point number $x$, $0 \le x \le 1$. 

**Goal:** Generate $0$ or $1$ with probability of $0$ equal to $x$ and probability of $1$ equal to $1 - x$.

> Useful Q# documentation: 
> * [`Math` namespace](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.math)
> * [`ArcCos` function](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.math.arccos)
> * [`Sqrt` function](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.math.sqrt)


### Solution



In [4]:
%kata T4_WeightedRandomBit

open Microsoft.Quantum.Math;

operation WeightedRandomBit (x : Double) : Int {
    // Set theta as 
    let theta = 2.0 *  ArcCos(Sqrt(x));

    // Allocate single qubit
    use q = Qubit();

    // Set qubit in superposition state which aligns with given probabilities
    Ry(theta, q);

    // Measuring state of qubit and return integer value of result
    return M(q) == Zero ? 0 | 1;
}

Testing generating zero with 0% probability...
Test passed for generating zero with 0% probability
Testing generating zero with 25% probability...
Test passed for generating zero with 25% probability
Testing generating zero with 50% probability...
Test passed for generating zero with 50% probability
Testing generating zero with 75% probability...
Test passed for generating zero with 75% probability
Testing generating zero with 100% probability...
Test passed for generating zero with 100% probability


Success!

## <span style="color:blue">Exercise 5</span>: Generate a random number between min and max

In exercise 3, we generated numbers in the range $[0, 2^N-1]$ $(1 \leq N \leq 10)$. Now let's create an operation that will return a random number in the range $[min, max]$. 

**Input:** 
Two integers $min$ and $max$ ($0 \leq min \leq max \leq 2^{10}-1$).

**Goal:** Generate a random number in the range $[min, max]$ with an equal probability of getting each of the numbers in this range.

> Useful Q# documentation: 
> * [`BitSizeI` function](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.math.bitsizei)

### Solution



In [5]:
%kata T5_RandomNumberInRange

open Microsoft.Quantum.Math;

operation RandomBit () : Int {
    // Allocate single qubit
    use q = Qubit();
    
    // Set qubit in superposition state
    H(q);
    
    // Measuring state of qubit and return integer value of result
    return (M(q) == Zero) ? 0 | 1;
}

operation RandomNumberInRange (min : Int, max : Int) : Int {
    // Set N as bitsize of max
    let N = BitSizeI(max);

    // Declare result variable with mutable binding
    mutable result = 0;
    
    repeat {
        set result = 0;

        for index in 1 .. N {
            set result = (result * 2) + RandomBit();
        }
    } until (result >= min and result <= max);

    return result;
}

Expecting only one Q# operation in code. Using the first one
Testing for min = 1 and max = 3...
Unexpected number generated. Expected values from 1 to 3, generated -1
Unexpected number generated. Expected values from 1 to 3, generated -1
Unexpected number generated. Expected values from 1 to 3, generated -1


Failed to generate sufficiently random integer
Try again!
